<a href="https://colab.research.google.com/github/Jayjoshina/Soc-project/blob/main/Soc_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
words = open('names.txt', 'r').read().splitlines()
import torch

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2,ch3 in zip(chs, chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3=stoi[ch3]

    xs.append([ix1,ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)


In [3]:
xs.shape[1]

2

In [4]:
#preparing dataset for bigram model from same dataset
x2, y2 = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]


    x2.append(ix1)
    y2.append(ix2)

x2 = torch.tensor(x2)
y2 = torch.tensor(y2)


In [5]:
#Manually doing one hot encoding
import numpy as np
x1=[]
for i in range(xs.shape[0]):
  b=np.zeros((27))
  b[xs[i][0].item()]=1
  b[xs[i][1].item()]=1
  x1.append(b)
x1=torch.tensor(x1, dtype=torch.float32)

x1

<ipython-input-5-1c574b732b8f>:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  x1=torch.tensor(x1, dtype=torch.float32)


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.]])

In [6]:
#one hot encoding for biagram model dataset
x3=torch.nn.functional.one_hot(x2, num_classes=27).float()
x3.dtype

torch.float32

In [7]:
#splitting data into test,dev,train
from sklearn.model_selection import train_test_split
train_ratio=0.8
test_ratio=0.1
validation_ratio=0.1

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, ys, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x1_val, x1_test, y1_val, y1_test = train_test_split(x1_test, y1_test, test_size=test_ratio/(test_ratio + validation_ratio))

x2_train, x2_test, y2_train, y2_test = train_test_split(x3, y2, test_size=1 - train_ratio)
x2_val, x2_test, y2_val, y2_test = train_test_split(x2_test, y2_test, test_size=test_ratio/(test_ratio + validation_ratio))


#y2_train=y2_train.view((y2_train.shape[0],1))
x2_test.shape

torch.Size([22815, 27])

In [8]:
#trigram model training
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)
for k in range(20):    #20 epochs

  # forward pass

  logits = x1_train @ W # predict log-counts
  loss= torch.nn.functional.cross_entropy(logits,y1_train)
  print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -10 * W.grad

4.12053108215332
3.846113443374634
3.638129472732544
3.4788100719451904
3.3536806106567383
3.2538304328918457
3.1734867095947266
3.108009099960327
3.0536139011383057
3.007500171661377
2.9677295684814453
2.932969093322754
2.902269124984741
2.87492299079895
2.85038685798645
2.8282315731048584
2.808112382888794
2.7897489070892334
2.7729105949401855
2.7574069499969482


In [9]:
#checking loss on validation set
with torch.no_grad():
 logits_val1 = x1_val @ W # predict log-counts
 loss1_val = torch.nn.functional.cross_entropy(logits_val1,y1_val)
print(loss1_val.item())

# loss almost saturates after learning rate of 10,so this is the min loss we can get and also loss of train and validation sets are almost equal so there is no overfitting

2.7451260089874268


In [10]:
with torch.no_grad():
 logits_test1 = x1_test @ W
 _,y1_pred=torch.max(logits_test1,1)
 acc=(y1_pred==y1_test).sum()
print(acc.item()/x1_test.shape[0])

0.2234346318580461


In [11]:
#bigram model training
g = torch.Generator().manual_seed(2147483647)
W2 = torch.randn((27, 27), generator=g, requires_grad=True)
for k in range(200):    #20 epochs

  # forward pass

  logits2 = x2_train @ W2 # predict log-counts
  #counts = logits.exp() # counts, equivalent to N
  #probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss2 = torch.nn.functional.cross_entropy(logits2,y2_train)


  # backward pass
  W2.grad = None # set to zero the gradient
  loss2.backward()

  # update
  W2.data += -10 * W2.grad
print(loss2.item())

2.5109479427337646


In [12]:
#checking loss on validation set
with torch.no_grad():
 logits_val2 = x2_val @ W2 # predict log-counts
 loss2_val = torch.nn.functional.cross_entropy(logits_val2,y2_val)
print(loss2_val.item())

# loss almost saturates after learning rate of 10,so this is the min loss we can get and also loss of train and validation sets are almost equal so there is no overfitting

2.5055418014526367


In [13]:
with torch.no_grad():
 logits_test2 = x2_test @ W2
 _,y2_pred=torch.max(logits_test2,1)
 acc=(y2_pred==y2_test).sum()
print(acc.item()/x2_test.shape[0])


0.225290379136533
